# Committor Analysis

(Note: some of this requires OpenPathSampling 0.9.1 or later)

In [1]:
import openpathsampling as paths

In [2]:
%%time
simulation_storage = paths.AnalysisStorage("committor_simulation.nc")
C_7eq = simulation_storage.volumes['C_7eq']
alpha_R = simulation_storage.volumes['alpha_R']

CPU times: user 1min 35s, sys: 10.2 s, total: 1min 45s
Wall time: 8min 6s


In [3]:
analyzer = paths.ShootingPointAnalysis(steps=None, states=[C_7eq, alpha_R])

In [4]:
%%time
# the shooting point snapshot for each
shooting_pts = [analyzer.step_key(step) for step in simulation_storage.steps]

CPU times: user 57.8 s, sys: 1.17 s, total: 59 s
Wall time: 2min 3s


In [5]:
%%time
# get the final states from each partial trajectory
final_states_list = [analyzer.analyze_single_step(step) for step in simulation_storage.steps]

CPU times: user 3min 25s, sys: 2.59 s, total: 3min 28s
Wall time: 5min 21s


In [6]:
# check that there's only one state per item in that list
for f in final_states_list:
    assert len(f) == 1
# flatten the list
final_states = [f[0] for f in final_states_list]

In [7]:
experiments = zip(shooting_pts, final_states)

In [8]:
output_storage = paths.Storage("committor_results.nc", "w")

In [9]:
output_storage.save(C_7eq)
output_storage.save(alpha_R)
output_storage.tag['experiments'] = experiments

In [10]:
output_storage.sync()
output_storage.close()